In [1]:
#step 1 import image
%matplotlib inline
import torchvision.datasets
import math
import torchvision.transforms as tvt
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wget
import zipfile
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as tfms
from torch.utils.data import DataLoader, Subset, Dataset, random_split
from torchvision.utils import make_grid
from PIL import Image
from time import time
from tqdm import tqdm
import random
from transformers import ViTModel

device = torch.device('cuda:1')

def seed_everything(seed):
    """
    Changes the seed for reproducibility. 
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(1024)
image_size = 64
batch_size = 1024
dataset = torchvision.datasets.CelebA("../celeba/datasets/",split='train', transform=tvt.Compose([
                                  tvt.Resize((image_size,image_size)),
                                  tvt.ToTensor(),
                                  tvt.Normalize(mean=[0.5, 0.5, 0.5],
                                                std=[0.5, 0.5, 0.5])                                  
                              ]))

test_dataset = torchvision.datasets.CelebA("../celeba/datasets/",split='test', transform=tvt.Compose([
                                  tvt.Resize((image_size,image_size)),
                                  tvt.ToTensor(),
                                  tvt.Normalize(mean=[0.5, 0.5, 0.5],
                                                std=[0.5, 0.5, 0.5])                                  
                              ]))

lengths = [int(len(dataset)*0.9), int(len(dataset)*0.1)]
if sum(lengths) != len(dataset):
    lengths[0] += len(dataset) - sum(lengths)
    
train_dataset, val_dataset = random_split(dataset, lengths)

training_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
valid_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
print('Done')

/home/shenyu/miniconda3/envs/DLcourse/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Done


In [2]:
from transformers import ViTConfig, ViTModel
configuration = ViTConfig(num_hidden_layers = 8, num_attention_heads = 8, 
                          intermediate_size = 768, image_size= 64, patch_size = 16)
model = ViTModel(configuration)
configuration = model.config
t = iter(test_data_loader)
img, label = next(t)
img
y = model(img)
y.last_hidden_state.shape

torch.Size([1024, 17, 768])

In [3]:
class VisionTransformer(nn.Module):
    def __init__(self, vit):
        super(VisionTransformer, self).__init__()
        self.vit = vit
        self.seq = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, 1),     
            nn.Sigmoid()
        )
    
    def forward(self, x):
        z = self.vit(x)
        m = z.last_hidden_state
        g = m[:,0]
        y = self.seq(g)
        return y 

In [4]:
import random
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import confusion_matrix
from collections import OrderedDict
import seaborn as sns

def seed_everything(seed):
    """
    Changes the seed for reproducibility. 
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def train_model():
    epoch = 30
    configuration = ViTConfig(num_hidden_layers = 8, num_attention_heads = 8, 
                          intermediate_size = 768, image_size= 64, patch_size = 16)
    vit = ViTModel(configuration)
    configuration = vit.config
    vit = vit.to(device)
    model = VisionTransformer(vit)
    model = model.to(device)
    
    criterion = nn.BCELoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-4)
    
    valid_acc = []
    valid_eod = []

    for epoches in range(epoch):
        with tqdm(training_data_loader, unit="batch") as tepoch:
            for train_input, attributes in tepoch:
                # Transfer data to GPU if possible. 
                train_input = train_input.to(device)
                sensitive, train_target = attributes[:,20], attributes[:,2]
               
                train_target = train_target.float().to(device)
                train_target = train_target.unsqueeze(1)
                optimizer.zero_grad()

                # Learner update step.
                outputs = model(train_input)
                loss = criterion(outputs, train_target)
                loss.backward()
                #logger_learner.add_values(logging_dict)
                optimizer.step()
                tepoch.set_description(f"epoch %2f " % epoches)
                tepoch.set_postfix(ut_loss = loss.item())
        # Reset the dataloader if out of data.
        #model.load_state_dict(torch.load(PATH), False)
        test_pred = []
        test_gt = []
        sense_gt = []
        female_predic = []
        female_gt = []
        male_predic = []
        male_gt = []
        
        
        
        



    # Evaluate on valdi set.
        for step, (test_input, attributes) in enumerate(test_data_loader):
            sensitive, test_target = attributes[:,20], attributes[:,2]
            test_input = test_input.to(device)
            test_target = test_target.to(device)

            gt = test_target.detach().cpu().numpy()
            sen = sensitive.detach().cpu().numpy()
            test_gt.extend(gt)
            sense_gt.extend(sen)

            # Todo: split according to sensitive attribute
            # Todo: combine all batch togather

            with torch.no_grad():
                test_pred_ = model(test_input)
                test_pred.extend(torch.round(test_pred_.squeeze(1)).detach().cpu().numpy())

        for i in range(len(sense_gt)):
            if sense_gt[i] == 0:
                female_predic.append(test_pred[i])
                female_gt.append(test_gt[i])
            else:
                male_predic.append(test_pred[i])
                male_gt.append(test_gt[i])
        female_CM = confusion_matrix(female_gt, female_predic)    
        male_CM = confusion_matrix(male_gt, male_predic) 
        female_dp = (female_CM[1][1]+female_CM[0][1])/(female_CM[0][0]+female_CM[0][1]+female_CM[1][0]+female_CM[1][1])
        male_dp = (male_CM[1][1]+male_CM[0][1])/(male_CM[0][0]+male_CM[0][1]+male_CM[1][0]+male_CM[1][1])
        female_TPR = female_CM[1][1]/(female_CM[1][1]+female_CM[1][0])
        male_TPR = male_CM[1][1]/(male_CM[1][1]+male_CM[1][0])
        female_FPR = female_CM[0][1]/(female_CM[0][1]+female_CM[0][0])
        male_FPR = male_CM[0][1]/(male_CM[0][1]+male_CM[0][0])
        
        valid_acc.append(accuracy_score(test_gt, test_pred))
        valid_eod.append(0.5*(abs(female_FPR-male_FPR)+ abs(female_TPR-male_TPR)))

        print('Female TPR', female_TPR)
        print('male TPR', male_TPR)
        print('DP',abs(female_dp - male_dp))
        print('EOP', abs(female_TPR - male_TPR))
        print('EoD',0.5*(abs(female_FPR-male_FPR)+ abs(female_TPR-male_TPR)))
        print('acc', accuracy_score(test_gt, test_pred))
        print('Trade off',accuracy_score(test_gt, test_pred)*(1-0.5*(abs(female_FPR-male_FPR)+ abs(female_TPR-male_TPR))) )
        
    return valid_acc,  valid_eod


seed_everything(1024)    
va, ve = train_model()

epoch 0.000000 : 100%|██████████████████████████████████████████████| 143/143 [02:14<00:00,  1.06batch/s, ut_loss=0.482]


Female TPR 0.8585921843687375
male TPR 0.47126436781609193
DP 0.5390445560463946
EOP 0.3873278165526456
EoD 0.3838897920661666
acc 0.7663059813645927
Trade off 0.4721289375194795


epoch 1.000000 : 100%|████████████████████████████████████████████████| 143/143 [02:15<00:00,  1.06batch/s, ut_loss=0.5]


Female TPR 0.8959168336673347
male TPR 0.6368861024033438
DP 0.5040434466181383
EOP 0.2590307312639909
EoD 0.3186324999819459
acc 0.7757739705440336
Trade off 0.5285871708886678


epoch 2.000000 : 100%|██████████████████████████████████████████████| 143/143 [02:15<00:00,  1.05batch/s, ut_loss=0.449]


...

Female TPR 0.8525801603206413
male TPR 0.6050156739811913
DP 0.43857838450629594
EOP 0.24756448633945005
EoD 0.27048656744620225
acc 0.75403266205791
Trade off 0.5500769555555437


epoch 28.000000 : 100%|████████████████████████████████████████████| 143/143 [02:15<00:00,  1.05batch/s, ut_loss=0.0315]


Female TPR 0.8351703406813628
male TPR 0.5606060606060606
DP 0.4473086561704293
EOP 0.2745642800753022
EoD 0.2810049639192382
acc 0.7579901813445546
Trade off 0.5449911777846912


epoch 29.000000 : 100%|████████████████████████████████████████████| 143/143 [02:15<00:00,  1.05batch/s, ut_loss=0.0179]


Female TPR 0.810370741482966
male TPR 0.5663531870428422
DP 0.41320976883198235
EOP 0.24401755444012374
EoD 0.24562188508974858
acc 0.7540827572387536
Trade off 0.5688635288920957


In [10]:
import matplotlib.pyplot as plt

# Sample data

# Create a basic plot
x = list(range(1, 31)) 
plt.plot(x, ve, marker='o')  # 'o' specifies marker style
plt.plot(x, va, marker='o')  # 'o' specifies marker style
#plt.xlabel('X-axis')
#plt.ylabel('Y-axis')
plt.grid(True)
plt.show()
